# División del conjunto de datos

A la hora de entrenar un modelo una parte fundamental es dividir el conjunto de 
datos en 3 secciones importantes:

1. Datos de entrenamiento (entre un 60% o 70% del conjunto)
2. Datos de validación (entre un 15% o 20% del conjunto)
   - Estos sirven para probar el modelo en caso de que el entrenamiento
     arroje valores de overfitting lo que nos permitirá reevaluar el
     modelo y/o aplicar técnicas como eleminiación de características o 
     regularización (penalizar ligeramnete los valores de peso para que 
     la función hipotesis no se ajuste tan perfectamente a los datos de
     entrenamiento)
3. Datos de prueba (entre un 15% o 20% del conjunto)
   - Estos datos se usan como prueba final para verificar que el modelo
     tiene un bajo nivel de error tanto en el conjunto de entrenamiento
     como en el de validamiento y no se ha sobre entrenado para ninguno
     de los dos.

Solamente así nos podemos asegurar que el modelo se entrenará correctamente y predecirá adecuadamente nuevos valores.

## 1. Lectura del conjunto de datos

Se continuará trabajando con los datos de flujos de red normales o anómalos *NSL-KDD*

In [1]:
import arff
import pandas as pd

In [2]:
# Funcion para cargar dataset en formato arff a un df de Pandas
def load_kdd_dataset(data_path):
    """Lectura del conjunto de datos NSL-KDD."""
    with open(data_path, 'r') as train_set:
        dataset = arff.load(train_set)
    attributes = [attr[0] for attr in dataset["attributes"]]
    return pd.DataFrame(dataset["data"], columns=attributes)

In [3]:
df = load_kdd_dataset('../datasets/NSL-KDD/KDDTrain+.arff')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  float64
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  object 
 4   src_bytes                    125973 non-null  float64
 5   dst_bytes                    125973 non-null  float64
 6   land                         125973 non-null  object 
 7   wrong_fragment               125973 non-null  float64
 8   urgent                       125973 non-null  float64
 9   hot                          125973 non-null  float64
 10  num_failed_logins            125973 non-null  float64
 11  logged_in                    125973 non-null  object 
 12  num_compromised              125973 non-null  float64
 13 

## 2. División del conjunto de datos

Separar el dataset actual en 3subconjuntos:

 * DF de entrenamiento
 * DF de prueba
 * DF de validación

In [19]:
# Separar el 60% para entrenamiento y 40% para train/validation
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.4, random_state=42)

In [20]:
len(train_set) # Tamaño de entrenamiento

75583

In [21]:
len(test_set) # Tamaño de pruebas

50390

In [22]:
# Separar el subconjunto de test 50/50 para obtener el set de validacion
val_set, test_set = train_test_split(test_set, test_size=0.5, random_state=42)

In [23]:
len(val_set)

25195

In [25]:
print(f'Tamaño del conjunto de entrenamiento: {len(train_set)}')
print(f'Tamaño del conjunto de validacion: {len(val_set)}')
print(f'Tamaño del conjunto de prueba: {len(test_set)}')

Tamaño del conjunto de entrenamiento: 75583
Tamaño del conjunto de validacion: 25195
Tamaño del conjunto de prueba: 25195


## 3. División del conjunto NO ALEATORIO

La función de sklearn *train_test_split* mezcla por defecto el conjunto de datos cada vez que ejecutamos volvemos a ingresar al script y lo ejecutamos. Esto hace de que manera indirecta el modelo terminará conociendo todo el conjunto sin importar que lo dividamos volviendo al problema del **overfitting**. Para solucionar esto la función incluye los parámetros *random_state* donde le damos una semilla de generación única y *shuffle* en false donde indicamos que no mezcle la información

In [28]:
# Evitar la mezcla de los datos
train_set, test_set = train_test_split( 
    df, test_size=0.4,
    random_state=42,
    shuffle=False 
)

El problema de hacer esto es que puede que algunas características como el *protocol_type* tenga coincidentemente valores de un tipo x en las primeras filas de todo el conjunto, si no hace una mezcla el modelo no será entrenado correctamente y podrá generara valores erroneos en el futuro.

Para solucionar este problema se puede identificar las características propensas a ocurrir esto (generalmente las caraterísticas categóricas/string) y usamos el parámetro **stratify** indicando el nombre de la columna que se va a encargar de esparcir homogéneamente a traves de todo el conjunto de datos general.

* **Aún así shuffle=False sigue siendo útil cuando se manejan grandes volumenes de datos**

In [29]:
train_set, test_set = train_test_split( 
    df,
    test_size=0.4,
    random_state=42,
    stratify=df['protocol_type'] 
)

## 4. Generar una función para división del conjunto de datos

Crear una función para reusar la división del conjunto de datos en los 3 subconjuntos mencionados

In [31]:
# Construcción de una función que realice el particionado completo
def train_val_test_split(df, rstate=42, shuffle=True, stratify=None):

    # Strat solo si le pasamos la columa a dispersar
    strat = df[stratify] if stratify else None 
    
    train_set, test_set = train_test_split(
        df,
        test_size=0.4,
        random_state=rstate, # Semilla de generación aleatoria única
        shuffle=shuffle, # Si se hace o no un shuffle
        stratify=strat # Columna a dispersar si la hay
    )

    # Se repite el proceso para obtener el validation_set
    strat = test_set[stratify] if stratify else None
    
    val_set, test_set = train_test_split(
        test_set,
        test_size=0.5,
        random_state=rstate,
        shuffle=shuffle,
        stratify=strat
    )
    
    return (train_set, val_set, test_set)

In [32]:
print(f'Longitud inicial del conjunto: {len(df)}')

Longitud inicial del conjunto: 125973


In [34]:
train_set, val_set, test_set = train_val_test_split(
    df, # Le pasamos el ds original
    stratify='protocol_type' # Pasamos el nombre de la columna que queramos dispersar
)

In [35]:
print(f'Tamaño del conjunto de entrenamiento: {len(train_set)}')
print(f'Tamaño del conjunto de validacion: {len(val_set)}')
print(f'Tamaño del conjunto de prueba: {len(test_set)}')

Tamaño del conjunto de entrenamiento: 75583
Tamaño del conjunto de validacion: 25195
Tamaño del conjunto de prueba: 25195
